In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from random import sample
from statistics import mean
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
def CI(stats):
    # plot scores
    plt.hist(stats)
    plt.show()
    
    # Confidence intervals
    alpha = 0.95
    # Lower and Upper
    p1 = ((1.0-alpha)/2.0) * 100  
    lower = max(0.0, np.percentile(stats, p1))
    p2 = (alpha+((1.0-alpha)/2.0)) * 100
    upper = min(1.0, np.percentile(stats, p2))
    
    print('%.1f confidence interval of MCC %.3f and %.3f' % (alpha*100, lower, upper))
    print('Average MCC %.3f' % np.mean(stats))

In [3]:
# Tanimoto coefficient
def tanimoto(fingerprint1, fingerprint2):
    fp1 = int(fingerprint1, 16)
    fp2 = int(fingerprint2, 16)
    fp1_count = bin(fp1).count('1')
    fp2_count = bin(fp2).count('1')
    both_count = bin(fp1 & fp2).count('1')  
    
    if (fp1_count + fp2_count - both_count) != 0:
        return float(both_count)/(fp1_count + fp2_count - both_count)
    else:
        return float(0)

In [4]:
def rogot_goldberg(fingerprint1, fingerprint2):
    sum1, sum2 = 0,0
    a,b,c,d = 0,0,0,0 
    for i in range(len(fingerprint1)):
        if fingerprint1[i] == "1" and fingerprint2[i] == "1":
            a += 1
            
        elif fingerprint1[i] == "1" and fingerprint2[i] == "0":
            b += 1
            
        elif fingerprint1[i] == "0" and fingerprint2[i] == "1":
            c += 1
           
        elif fingerprint1[i] == "0" and fingerprint2[i] == "0":
            d += 1
    try:
        sum1 = a/(2*a+b+c)
    except ZeroDivisionError:
        sum1 = 0 
        
    try:
        sum2 = d/(2*d+b+c)
    except ZeroDivisionError:
        sum2 = 0 

    return sum1+sum2

In [5]:
def get_opt_thres(fpr, tpr, thresholds):
    # Calculate the G-mean
    gmean = np.sqrt(tpr*(1-fpr))
    # Maximal G-mean corresponding index
    index = np.argmax(gmean)
    # get optimal threshold corrsponding to maximal G-mean
    thresholdOpt = round(thresholds[index], ndigits=4)
    return thresholdOpt

## Similarity search performance evaluation
Metrics: AUC and EF10

In [6]:
path = '/Users/hek/Research/Cheminformatics/Project_1_NPS/Stimulant vs. Hallucinogens/ChEMBL Dataset ML results/'

In [7]:
NPS_active = pd.read_csv(path+"Drugs Raman or SERS in literature _ Paper 1.csv",dtype="object")
NPS_active = NPS_active.astype({"Pharm class label": "float64"})
NPS_inactive = pd.read_csv(path+"Decoys Drugs Raman or SERS in literature _ Paper 1.csv",dtype="object")
print(NPS_active['Pharm class'].unique())

['Depressants_Opioids' 'Stimulants' 'Cannabinoids'
 'Serotonergic psychedelics' 'Depressants_Benzodiazepines']


### Part I. Similarity search for each Pharm Category
* 1 == 'Depressants_Opioids'
* 2 == 'Stimulants'
* 3 == 'Cannabinoids'
* 4 == 'Serotonergic psychedelics'
* 5 == 'Depressants_Benzodiazepines'

#### 1. Assemble query list (training) and test list molecules for each pharm category
##### Repeat similarity search for 50 times
##### Each iteration: 
* random sample 10 actives, and 20% of the decoys as query list
* remaining active and decoys as test set

In [8]:
def actives_query_test_split(pharm_class):
    # Subset actives by pharm class label according to category, reindex dataframe
    actives = NPS_active[NPS_active['Pharm class label']==pharm_class]
    #print("Total of", actives.shape[0], "actives in pharm class = ",pharm_class)
    actives.reset_index(drop=True,inplace=True)
    actives_tmp = actives.copy()
    # Random sample 10 active by index
    random10_active = sample(range(actives.shape[0]), 10)
    # subset by random active index
    actives_query = actives.iloc[random10_active,:]
    #print(actives_query.Name)

    # drop sampled actives by index
    actives_tmp.drop(random10_active,inplace=True)
    #print("Number actives added to test list", P_total = actives_tmp.shape[0])
    P_total = actives_tmp.shape[0]
    n_size = P_total*10
    #print("Number of decoys to be added to test list",n_size)
    return actives_query, actives_tmp,P_total, n_size

In [9]:
def decoys_query_test_split(n_size):
    decoys_tmp = NPS_inactive.copy()
    decoys_tmp['Pharm class label']=np.zeros(len(decoys_tmp))
    #print(decoys_tmp.shape[0])
    # Random sample 100 decoys by index
    random_decoy = sample(range(decoys_tmp.shape[0]),100)
    # Subset by random decoy index
    decoys_query = decoys_tmp.iloc[random_decoy,:]
    #print("Number of decoys added to query", decoys_query.shape[0])
    # Drop sampled qeury decoys by index, and reset
    decoys_tmp.drop(random_decoy,inplace=True)
    decoys_tmp.reset_index(drop=True, inplace=True)
    
    # Random sample n_size = (query actives)*10
    random_decoy_test = sample(range(decoys_tmp.shape[0]),n_size)
    decoys_test = decoys_tmp.iloc[random_decoy_test,:]
    #print("Number decoys added to test list", decoys_test.shape[0])
    
    return decoys_query, decoys_test

In [10]:
def merge_query_test(pharm_class):
    actives_query, actives_test, P_total,n_size = actives_query_test_split(pharm_class)
    decoys_query, decoys_test = decoys_query_test_split(n_size)
    
    df_query = pd.concat([actives_query,decoys_query],axis=0)
    df_query.reset_index(drop=True, inplace=True)
    #df_query = df_query[['Name','Pharm class label','maccsfp','morganfp']]
    #print(df_query.shape[0])
    df_test = pd.concat([actives_test,decoys_test],axis=0)
    df_test.reset_index(drop=True, inplace=True)
    #df_test = df_test[['Name','Pharm class label','maccsfp','morganfp']]
    #print(df_test.shape[0])
    N_total = df_test.shape[0]
    #print("Test set P_total/N_total ratio", P_total/N_total)
    return df_query, df_test, P_total, N_total

In [11]:
def df_test_search_result(df_query, df_test,fingerprint):
    df_test_search = df_test.copy()
    # For each molecule in the test set, calculate its similarity to query molecules, and only the highest similarity value was considered
    top_tani = []
    for test in range(df_test.shape[0]):
        #print(df_test.loc[test,"Name"])
        #print(len(df_test.loc[0,fingerprint]))
        tani = []
        for query in (range(df_query.shape[0])):
            #print(df_query.loc[query,"Name"])
            tani.append(tanimoto(df_test.loc[test,fingerprint],df_query.loc[query,fingerprint]))
        tani = sorted(tani,reverse=True)
        #print(tani[0])
        top_tani.append(tani[0])
    df_test_search["top_tani"] = top_tani
    
    df_test_search.sort_values(["top_tani"],ascending=False,inplace=True)
    df_test_search.reset_index(drop=True, inplace=True)
    
    #print(df_test.head())
    return df_test_search

In [12]:
def df_test_search_result_Phfp(df_query, df_test,fingerprint):
    df_test_search = df_test.copy()
    # For each molecule in the test set, calculate its similarity to query molecules, and only the highest similarity value was considered
    top_tani = []
    for test in range(df_test.shape[0]):
        #print(df_test.loc[test,"Name"])
        #print(len(df_test.loc[0,fingerprint]))
        tani = []
        for query in (range(df_query.shape[0])):
            #print(df_test.loc[test,fingerprint])
            tani.append(rogot_goldberg(df_test.loc[test,fingerprint],df_query.loc[query,fingerprint]))
        tani = sorted(tani,reverse=True)
        #print(tani[0])
        top_tani.append(tani[0])
    df_test_search["top_tani"] = top_tani
    
    df_test_search.sort_values(["top_tani"],ascending=False,inplace=True)
    df_test_search.reset_index(drop=True, inplace=True)
    
    #print(df_test.head())
    return df_test_search

In [13]:
#Enrichment factor of top 10%
def EF10(df_test_search, P_total, N_total):
    # Count actives of the top5% of the sorted dataset
    N_10 = int(N_total*0.1)
    df_test_top = df_test_search.head(N_10)
    
    P_10 = 0 
    for label in df_test_top["Pharm class label"]:
        if label != 0:
            P_10 += 1
    
    # Calculate EF10
    EF10 = (P_10/N_10)/(P_total/N_total)
    return EF10

In [14]:
def EF10_AUC_average(pharm_class,n_iteration,fingerprint):
    print("Repeat similarity search ",n_iteration,"times","for Pharm class", pharm_class)
    EF10_total, AUC_total,optimal_thres = [],[],[]
    for iteration in range(n_iteration):
        df_query, df_test, P_total, N_total = merge_query_test(pharm_class)
        df_test_search = df_test_search_result(df_query, df_test, fingerprint)
        
        
        EF10_score = EF10(df_test_search, P_total, N_total)
        EF10_total.append(EF10_score)
        
        
        AUC_score = roc_auc_score(np.array(df_test_search['Pharm class label']), np.array(df_test_search['top_tani']))
        AUC_total.append(AUC_score)
        
        fpr, tpr, thres = roc_curve(np.array(df_test_search['Pharm class label']), np.array(df_test_search['top_tani']), pos_label=pharm_class)
        thresOpt = get_opt_thres(fpr, tpr, thres)
        optimal_thres.append(thresOpt)
        
    print("Average EF10: %.2f" %(mean(EF10_total)), ", Average AUC: %0.2f" %(mean(AUC_total)), ", Average opt thres: %0.2f" %mean(optimal_thres))
    print("Std EF10: %.2f" %np.std(EF10_total), ", Std AUC: %.2f" %np.std(AUC_total), ", Std opt thres: %0.2f" %np.std(optimal_thres))
    return EF10_total, AUC_total, optimal_thres

In [15]:
def EF10_AUC_average_Phfp(pharm_class,n_iteration,fingerprint):
    print("Repeat similarity search ",n_iteration,"times","for Pharm class", pharm_class)
    EF10_total, AUC_total,optimal_thres = [],[],[]
    for iteration in range(n_iteration):
        df_query, df_test, P_total, N_total = merge_query_test(pharm_class)
        df_test_search = df_test_search_result_Phfp(df_query, df_test, fingerprint)
        
        
        EF10_score = EF10(df_test_search, P_total, N_total)
        EF10_total.append(EF10_score)
        
        
        AUC_score = roc_auc_score(np.array(df_test_search['Pharm class label']), np.array(df_test_search['top_tani']))
        AUC_total.append(AUC_score)
        
        fpr, tpr, thres = roc_curve(np.array(df_test_search['Pharm class label']), np.array(df_test_search['top_tani']), pos_label=pharm_class)
        thresOpt = get_opt_thres(fpr, tpr, thres)
        optimal_thres.append(thresOpt)
        
    print("Average EF10: %.2f" %(mean(EF10_total)), ", Average AUC: %0.2f" %(mean(AUC_total)), ", Average opt thres: %0.2f" %mean(optimal_thres))
    print("Std EF10: %.2f" %np.std(EF10_total), ", Std AUC: %.2f" %np.std(AUC_total), ", Std opt thres: %0.2f" %np.std(optimal_thres))
    return EF10_total, AUC_total, optimal_thres

## Evaluate similarity search using moleculer fingerprints

In [32]:
df_result = pd.DataFrame()

In [39]:
# Repeat similarity search for all four pharm classes
Pharmclass = "Serotonergic psychedelics"
Pharmclass_label = 4

fingerprint1 = "RF_p5_maccs"
fingerprint2 = "RF_p5_morgan"

In [40]:
print('Search using', fingerprint1, "Tanimoto similarity score")
df_result[Pharmclass+'_'+fingerprint1+'_'+"EF10"], df_result[Pharmclass+'_'+fingerprint1+'_'+"AUC"],df_result[Pharmclass+'_'+fingerprint1+'_'+"thres"] = EF10_AUC_average(Pharmclass_label,50,fingerprint1)
print("**"*10)
print('Search using', fingerprint2, "Tanimoto similarity score")
df_result[Pharmclass+'_'+fingerprint2+'_'+"EF10"], df_result[Pharmclass+'_'+fingerprint2+'_'+"AUC"] , df_result[Pharmclass+'_'+fingerprint2+'_'+"thres"] = EF10_AUC_average(Pharmclass_label,50,fingerprint2)

Search using RF_p5_maccs Tanimoto similarity score
Repeat similarity search  50 times for Pharm class 4
Average EF10: 8.84 , Average AUC: 0.97 , Average opt thres: 0.49
Std EF10: 0.58 , Std AUC: 0.01 , Std opt thres: 0.05
********************
Search using RF_p5_morgan Tanimoto similarity score
Repeat similarity search  50 times for Pharm class 4
Average EF10: 3.02 , Average AUC: 0.91 , Average opt thres: 0.36
Std EF10: 1.41 , Std AUC: 0.01 , Std opt thres: 0.08


In [19]:
df_result.head()

,Depressants_Benzodiazepines_maccsfp_EF10,Depressants_Benzodiazepines_maccsfp_AUC,Depressants_Benzodiazepines_maccsfp_thres,Depressants_Benzodiazepines_morganfp_EF10,Depressants_Benzodiazepines_morganfp_AUC,Depressants_Benzodiazepines_morganfp_thres
0,11.000000,1.000000,0.9070,7.333333,0.988889,0.4286
1,3.666667,0.877778,0.6719,7.333333,0.977778,0.4688
2,7.333333,0.944444,0.7500,11.000000,1.000000,0.4242
3,7.333333,0.888889,0.9535,11.000000,1.000000,0.5323
4,11.000000,1.000000,0.8125,7.333333,0.988889,0.4151


In [92]:
#df_result.to_csv(path+"Structural similarity search Tanimoto EF10 and AUC.csv",index=False)

## Evaluate similarity search using pharmacological fingerprints

In [23]:
df_result = pd.DataFrame()

In [30]:
# Repeat similarity search for all four pharm classes
Pharmclass = "Cannabinoids"
Pharmclass_label = 3

fingerprint3 = "RF_p5_morgan"
fingerprint4 = "RF_p6_morgan"
fingerprint5 = "RF_p7_morgan"

##### Use Rogot-Goldberg

In [31]:
print('Search using', fingerprint3, "Rogot-Goldberg similarity score")
df_result[Pharmclass+'_'+fingerprint3+'_'+"EF10"], df_result[Pharmclass+'_'+fingerprint3+'_'+"AUC"],df_result[Pharmclass+'_'+fingerprint3+'_'+"thres"] = EF10_AUC_average_Phfp(Pharmclass_label,50,fingerprint3)
print("**"*10)
print('Search using', fingerprint4, "Rogot-Goldberg similarity score")
df_result[Pharmclass+'_'+fingerprint4+'_'+"EF10"], df_result[Pharmclass+'_'+fingerprint4+'_'+"AUC"],df_result[Pharmclass+'_'+fingerprint4+'_'+"thres"] = EF10_AUC_average_Phfp(Pharmclass_label,50,fingerprint4)
print("**"*10)
print('Search using', fingerprint5, "Rogot-Goldberg similarity score")
df_result[Pharmclass+'_'+fingerprint5+'_'+"EF10"], df_result[Pharmclass+'_'+fingerprint5+'_'+"AUC"],df_result[Pharmclass+'_'+fingerprint5+'_'+"thres"] = EF10_AUC_average_Phfp(Pharmclass_label,50,fingerprint5)
print("**"*10)

Search using RF_p5_morgan Rogot-Goldberg similarity score
Repeat similarity search  50 times for Pharm class 3
Average EF10: 3.29 , Average AUC: 0.92 , Average opt thres: 0.77
Std EF10: 1.99 , Std AUC: 0.02 , Std opt thres: 0.06
********************
Search using RF_p6_morgan Rogot-Goldberg similarity score
Repeat similarity search  50 times for Pharm class 3
Average EF10: 7.88 , Average AUC: 0.97 , Average opt thres: 0.80
Std EF10: 0.97 , Std AUC: 0.01 , Std opt thres: 0.05
********************
Search using RF_p7_morgan Rogot-Goldberg similarity score
Repeat similarity search  50 times for Pharm class 3
Average EF10: 3.69 , Average AUC: 0.93 , Average opt thres: 0.79
Std EF10: 1.74 , Std AUC: 0.02 , Std opt thres: 0.04
********************


In [120]:
df_result.head(2)

,Stimulants_RF_p5_maccs_EF10,Stimulants_RF_p5_maccs_AUC,Stimulants_RF_p5_maccs_thres,Stimulants_RF_p6_maccs_EF10,Stimulants_RF_p6_maccs_AUC,Stimulants_RF_p6_maccs_thres,Stimulants_RF_p7_maccs_EF10,Stimulants_RF_p7_maccs_AUC,Stimulants_RF_p7_maccs_thres,Cannabinoids_RF_p5_maccs_EF10,...,Depressants_Opioids_RF_p7_maccs_thres,Depressants_Benzodiazepines_RF_p5_maccs_EF10,Depressants_Benzodiazepines_RF_p5_maccs_AUC,Depressants_Benzodiazepines_RF_p5_maccs_thres,Depressants_Benzodiazepines_RF_p6_maccs_EF10,Depressants_Benzodiazepines_RF_p6_maccs_AUC,Depressants_Benzodiazepines_RF_p6_maccs_thres,Depressants_Benzodiazepines_RF_p7_maccs_EF10,Depressants_Benzodiazepines_RF_p7_maccs_AUC,Depressants_Benzodiazepines_RF_p7_maccs_thres
0,8.768116,0.991736,0.7838,7.971014,0.970723,0.7418,5.739130,0.860015,0.6581,8.25,...,0.8218,7.333333,0.977778,0.9263,11.0,1.0,0.831,7.333333,0.816667,1.0000
1,9.565217,0.991358,0.7831,8.130435,0.968191,0.5988,6.536232,0.856286,0.6566,8.25,...,0.7058,11.000000,1.000000,0.8567,11.0,1.0,0.831,7.333333,0.938889,0.6925


In [121]:
df_result.to_csv(path+"Ph-fp maccsfp similarity search Rogot-Goldberg EF10 and AUC.csv",index=False)